[View in Colaboratory](https://colab.research.google.com/github/rizkyfalih/NLP-Exploration/blob/master/testPost.ipynb)

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Indonesian_Manually_Tagged_Corpus_ID.tsv to Indonesian_Manually_Tagged_Corpus_ID.tsv
User uploaded file "Indonesian_Manually_Tagged_Corpus_ID.tsv" with length 2764611 bytes


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 27 06:48:18 2018

@author: rizkyfalih
"""

# Import libraries
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from nltk import word_tokenize

def read_dataset(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sent.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }

def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y
 
def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    #return zip(sentence, tags)
    return tags

sentences,tags = read_dataset('Indonesian_Manually_Tagged_Corpus_ID.tsv')
training_sentences = sentences[:1000]
test_sentences = sentences[1000:1020]
training_tags = tags[:1020]
test_tags = tags[1000:1020]

X, y = transform_to_dataset(training_sentences, training_tags)
X_test, y_test = transform_to_dataset(test_sentences, test_tags)

clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y)   

print("Accuracy:")
print(clf.score(X_test, y_test))

Accuracy:
0.9323529411764706


In [10]:
print(X_test[2])

{'word': 'Efek', 'prefix-1': 'E', 'prefix-2': 'Ef', 'prefix-3': 'Efe', 'suffix-1': 'k', 'suffix-2': 'ek', 'suffix-3': 'fek', 'prev_word': 'Pemeringkat', 'next_word': 'Indonesia'}
